<a href="https://colab.research.google.com/github/lauraluebbert/test_gget_alphafold/blob/main/test_gget_alphafold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [gget alphafold](https://github.com/pachterlab/gget) test

___

Install and import gget:

In [ ]:
!uv pip install gget
import gget

Install OpenMM:  

In [ ]:
import os, sys, subprocess, site, pathlib, textwrap
%config InlineBackend.figure_format='retina'

# For use with Amber relax
!uv pip install "numpy<1.24" --force-reinstall

# Force JAX to ignore CUDA plugins and use CPU
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

# Make sure GPU plugins are gone and JAX is on a matching CPU wheel
subprocess.run(f"{sys.executable} -m pip uninstall -y jax-cuda12-plugin jax-cuda12-pjrt jax_plugins", shell=True, check=False)
subprocess.run(f"{sys.executable} -m pip install -U 'jax==0.4.26' 'jaxlib==0.4.26'", shell=True, check=True)

# Install OpenMM 8
!pip install "openmm>=8,<9"

# Create a lightweight 'simtk' shim that re-exports OpenMM 8 symbols
site_pkgs = site.getsitepackages()[0]  # e.g., /usr/local/lib/python3.x/dist-packages
shim_root = pathlib.Path(site_pkgs) / "simtk"
(shim_root / "openmm" / "app").mkdir(parents=True, exist_ok=True)

# simtk/__init__.py
(shim_root / "__init__.py").write_text("")

# simtk/openmm/__init__.py -> re-export openmm 8 API
(shim_root / "openmm" / "__init__.py").write_text(textwrap.dedent("""\
    # Compatibility shim for legacy imports: `import simtk.openmm as openmm`
    from openmm import *  # re-export core openmm symbols
    from openmm import unit as unit  # provide `simtk.openmm.unit` attribute
    try:
        # make `from simtk.openmm import app` work via attribute
        from openmm import app as app
    except Exception:
        pass
"""))

# simtk/openmm/app/__init__.py -> re-export app layer
(shim_root / "openmm" / "app" / "__init__.py").write_text(textwrap.dedent("""\
    # Compatibility shim for: `from simtk.openmm.app import *`
    from openmm.app import *
"""))

# simtk/unit.py -> legacy `from simtk import unit`
(shim_root / "unit.py").write_text("from openmm.unit import *\n")

# Test the legacy imports
try:
    import simtk.openmm as openmm
    from simtk.openmm import app
    from simtk import unit
    print("Shim OK. OpenMM version:", openmm.version.version)
    # quick smoke test
    print("Has app.PDBFile:", hasattr(app, "PDBFile"))
    print("Has unit.nanometer:", hasattr(unit, "nanometer"))
except Exception as e:
    print("Shim/import failed:", repr(e))

Install third-part dependencies and download AlphaFold model parameters using `gget setup` (this might take a few minutes):

In [ ]:
gget.setup("alphafold")

Predict the 3D structure of CASP14 target [T1024](https://predictioncenter.org/casp14/target.cgi?id=8&view=all):

In [ ]:
gget.alphafold(
    "MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH",
    relax=True,
    out=None
    )